In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.append('../kaggle_prediction_library/') 
import preprocess
import feature_engineering
import submission
from sklearn.model_selection import train_test_split

# from hyperopt import tpe, fmin, Trials
# import hyperopt.hp as hp

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import log_loss
from sklearn.feature_selection import chi2
from sklearn.metrics import r2_score
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import brier_score_loss


In [3]:
tmp_538 = pd.read_csv('../data/fivethirtyeight_2016.csv')

### Create data

In [4]:
regular_season_results = pd.read_csv('../data/MRegularSeasonDetailedResults.csv')
detailed_tourney_results = pd.read_csv('../data/MNCAATourneyDetailedResults.csv')
rankings = pd.read_csv('../data/MMasseyOrdinals.csv')
seeds = pd.read_csv('../data/MNCAATourneySeeds.csv')

kp_rankings = pd.read_csv('../data/kenpom_pre_tourney_snapshot.csv')

regular_season_results_w = pd.read_csv('../data/WRegularSeasonDetailedResults.csv')
detailed_tourney_results_w = pd.read_csv('../data/WNCAATourneyDetailedResults.csv')

sub_df = pd.read_csv('../data/SampleSubmission2023.csv')
mteams = pd.read_csv('../data/MTeams.csv')
wteams = pd.read_csv('../data/WTeams.csv')

seeds_w = pd.read_csv('../data/WNCAATourneySeeds.csv')

# _538_df = pd.read_csv('../data/all_538.csv')

M538 = pd.read_csv('../data/M538.csv')
W538 = pd.read_csv('../data/W538.csv')




In [5]:
to_predict_mens, to_predict_womens, regular_season_games, regular_season_games_w = preprocess.full_setup(detailed_tourney_results, regular_season_results,
               detailed_tourney_results_w, regular_season_results_w,
               sub_df, mteams)

In [6]:
to_predict_womens = feature_engineering.TournamentSeed(tourney_seeds=seeds_w).add(to_predict_womens)
to_predict_womens = feature_engineering.Efficiency(games=regular_season_games_w, away_bonus=0).add(to_predict_womens)
to_predict_womens = feature_engineering.FiveThirtyEight(fivethirtyeight_df=W538).add(to_predict_womens)

In [7]:
to_predict_mens = feature_engineering.SeasonStats(regular_season_games).add(to_predict_mens)
to_predict_mens = feature_engineering.FiveThirtyEight(fivethirtyeight_df=M538).add(to_predict_mens)
to_predict_mens = feature_engineering.PreSeasonAPRankings(rankings_df=rankings).add(to_predict_mens)
to_predict_mens = feature_engineering.TournamentSeed(tourney_seeds=seeds).add(to_predict_mens)
to_predict_mens = feature_engineering.Efficiency(games=regular_season_games, away_bonus=0).add(to_predict_mens)
to_predict_mens = feature_engineering.FinalRanking(rankings_df=rankings, system='SAG').add(to_predict_mens)
to_predict_mens = feature_engineering.Kenpom(kp_snapshot=kp_rankings).add(to_predict_mens)

# this one takes 3 minutes to run
# to_predict_mens = feature_engineering.TeamQuality(games=regular_season_games).add(to_predict_mens)


In [8]:
seed_round = pd.read_csv("../data/MNCAATourneySeedRoundSlots.csv")
seeds = pd.read_csv("../data/MNCAATourneySeeds.csv")

to_predict_mens = feature_engineering.RoundNumber(seeds, seed_round).add(to_predict_mens)


In [9]:
def add_team_names(df, team_names):
    team_id1 = team_names.rename(columns = {'TeamID': 'Team1', 
                                            'TeamName':'Team1_Name'}).copy()
    team_id2 = team_names.rename(columns = {'TeamID': 'Team2', 'TeamName':'Team2_Name'}).copy()
    
    df = df.merge(team_id1[['Team1', 'Team1_Name']], on = ['Team1'], how = 'left')
    df = df.merge(team_id2[['Team2', 'Team2_Name']], on = ['Team2'], how = 'left')

    return df

tmp = add_team_names(to_predict_mens, mteams)

In [12]:
first_round_df = tmp[tmp.GameRound == 1].copy()
other_rounds_df = tmp[tmp.GameRound > 1].copy()

In [13]:
feature_engineering.FirstRoundOpponentQuality(first_round_df, other_rounds_df).add()


,type,ID,Pred,Season,Team1,Team2,Outcome,Gender,margin,t1_FGM,...,t2_sos_opp_o,t2_sos_opp_o_rank,t2_sos_opp_d,t2_sos_opp_d_rank,t2_ncsos_adjem_rank,GameRound,Team1_Name,Team2_Name,round1_opponent_rank,round1_opponent_quality
0,Historical,2003_1112_1211,None,2003,1112,1211,1,M,1,30.321429,...,NaN,NaN,NaN,NaN,NaN,2.0,Arizona,Gonzaga,72.542565,0.290757
1,Historical,2003_1163_1390,None,2003,1163,1390,1,M,11,29.533333,...,NaN,NaN,NaN,NaN,NaN,2.0,Connecticut,Stanford,85.258089,0.683338
2,Historical,2003_1181_1141,None,2003,1181,1141,1,M,26,27.366667,...,NaN,NaN,NaN,NaN,NaN,2.0,Duke,C Michigan,77.955307,0.457871
3,Historical,2003_1242_1113,None,2003,1242,1113,1,M,32,30.233333,...,NaN,NaN,NaN,NaN,NaN,2.0,Kansas,Arizona St,76.909200,0.425573
4,Historical,2003_1266_1281,None,2003,1266,1281,1,M,9,27.214286,...,NaN,NaN,NaN,NaN,NaN,2.0,Marquette,Missouri,79.972665,0.520155
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3517,Prediction,2023_1436_1452,0.5,2023,1436,1452,None,M,None,26.187500,...,NaN,NaN,NaN,NaN,NaN,5.0,Vermont,West Virginia,NaN,NaN
3518,Prediction,2023_1436_1462,0.5,2023,1436,1462,None,M,None,26.187500,...,NaN,NaN,NaN,NaN,NaN,6.0,Vermont,Xavier,NaN,NaN
3519,Prediction,2023_1438_1452,0.5,2023,1438,1452,None,M,None,24.031250,...,NaN,NaN,NaN,NaN,NaN,3.0,Virginia,West Virginia,NaN,NaN
3520,Prediction,2023_1438_1462,0.5,2023,1438,1462,None,M,None,24.031250,...,NaN,NaN,NaN,NaN,NaN,6.0,Virginia,Xavier,NaN,NaN


In [62]:
first_round_opp = first_round_df.rename(columns = {"t2_final_rank":"round1_opponent_rank"})

In [63]:
tmp2 = other_rounds_df.merge(first_round_opp[["Season", "Team1", "round1_opponent_rank"]], how = "left", on = ["Season", "Team1"])

In [64]:
tmp3 = tmp2[~tmp2.round1_opponent_rank.isna()].copy()

In [65]:
tmp3["round1_opponent_quality"] = (tmp3["round1_opponent_rank"] - tmp3["round1_opponent_rank"].min()) / (tmp3["round1_opponent_rank"].max() - tmp3["round1_opponent_rank"].min())

In [77]:
tmp3.sort_values(by=["Season", "round1_opponent_quality", "Team1_Name"], 
                 ascending=[True, False, False])[[
    'Season', 'Team1_Name', 'Team2_Name', 'round1_opponent_rank', 'round1_opponent_quality',
    't1_Seed', 't2_Seed'
]].head(5)

,Season,Team1_Name,Team2_Name,round1_opponent_rank,round1_opponent_quality,t1_Seed,t2_Seed
9,2003,Butler,Louisville,87.939874,0.766136,12.0,4.0
610,2003,Butler,Oklahoma,87.939874,0.766136,12.0,1.0
591,2003,C Michigan,Duke,87.404062,0.749593,11.0,3.0
8,2003,Auburn,Wake Forest,86.912819,0.734426,10.0,2.0
611,2003,Auburn,Syracuse,86.912819,0.734426,10.0,3.0


In [ ]:
# I think this is working fine
# we just get more matchups for 2023 since every team can play everyone

In [9]:
to_predict_mens.to_csv("to_predict_mens.csv")

In [11]:
to_predict_mens[(to_predict_mens.Team1 == 1104)
                & (to_predict_mens.Season == 2004)]

,type,ID,Pred,Season,Team1,Team2,Outcome,Gender,t1_FGM,t1_FGA,...,t2_adjt_rank,t2_luck,t2_luck_rank,t2_sos_adjem,t2_sos_adjem_rank,t2_sos_opp_o,t2_sos_opp_o_rank,t2_sos_opp_d,t2_sos_opp_d_rank,t2_ncsos_adjem_rank
65,Historical,2004_1104_1356,None,2004,1104,1356,1,M,24.896552,55.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,Historical,2004_1104_1390,None,2004,1104,1390,1,M,24.896552,55.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
113,Historical,2004_1104_1393,None,2004,1104,1393,1,M,24.896552,55.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1369,Historical,2004_1104_1163,None,2004,1104,1163,0,M,24.896552,55.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
regular_season_games.head()

,TourneyGame,Season,DayNum,Team1,Team2,Team1_score,Team2_score,WLoc,NumOT,Team1_FGM,...,Team2_FTA,Team2_OR,Team2_DR,Team2_Ast,Team2_TO,Team2_Stl,Team2_Blk,Team2_PF,Outcome,Loc
0,0,2003,10,1104,1328,68,62,N,0,27,...,22,10,22,8,18,9,2,20,1,N
1,0,2003,10,1272,1393,70,63,N,0,26,...,20,20,25,7,12,8,6,16,1,N
2,0,2003,11,1266,1437,73,61,N,0,24,...,23,31,22,9,12,2,5,23,1,N
3,0,2003,11,1296,1457,56,50,N,0,18,...,15,17,20,9,19,4,3,23,1,N
4,0,2003,11,1400,1208,77,71,N,0,30,...,27,21,15,12,10,7,1,14,1,N


### Predictions

In [21]:
def make_predictions(to_predict_df, features, pred_suffix='', prediction_season=2023, min_train_season=2015):

    to_predict_df_train = to_predict_df[ (to_predict_df.Season >= min_train_season) 
                                        & (to_predict_df.Season < prediction_season) ].copy()

    to_predict_df_test = to_predict_df[ (to_predict_df.Season == prediction_season) ].copy()

    model = ModelPrediction()
    model.fit(to_predict_df_train[features], to_predict_df_train['Outcome'].astype(int))
    preds = model.predict_proba(to_predict_df_test[features])
    to_predict_df_test['model_preds_' + pred_suffix] = preds[:,1]

    return to_predict_df_test[['type', 'ID', 'Team1', 'Team2', 'model_preds_' + pred_suffix]]


def add_team_names(df, team_names):
    team_id1 = team_names.rename(columns = {'TeamID': 'Team1', 
                                            'TeamName':'Team1_Name'}).copy()
    team_id2 = team_names.rename(columns = {'TeamID': 'Team2', 'TeamName':'Team2_Name'}).copy()
    
    df = df.merge(team_id1[['Team1', 'Team1_Name']], on = ['Team1'], how = 'left')
    df = df.merge(team_id2[['Team2', 'Team2_Name']], on = ['Team2'], how = 'left')

    return df


In [24]:
# 538 model
features_538 = ['t1_team_rating_538', 
        't2_team_rating_538']

# Only matchups in 538 dataset 
to_predict_538_mens = to_predict_mens[(~to_predict_mens.t1_team_rating_538.isna())
                    & (~to_predict_mens.t2_team_rating_538.isna())]

to_predict_538_womens = to_predict_womens[(~to_predict_womens.t1_team_rating_538.isna())
                    & (~to_predict_womens.t2_team_rating_538.isna())]

preds_538_mens = make_predictions(to_predict_538_mens, features_538, pred_suffix='538_mens')
preds_538_womens = make_predictions(to_predict_538_womens, features_538, pred_suffix='538_womens')

In [25]:
# skyler model
features_sky_mens = [
    'seed_diff', 
            't1_adj_margin', 't2_adj_margin',
                't1_final_rank', 't2_final_rank',
                't1_OrdinalRank', 't2_OrdinalRank']

features_sky_womens = [
    'seed_diff', 
            't1_adj_margin', 't2_adj_margin']

# Only tournament matchups
to_predict_sky_mens = to_predict_mens[(~to_predict_mens.seed_diff.isna())
                    & (~to_predict_mens.seed_diff.isna())].copy()

to_predict_sky_womens = to_predict_womens[(~to_predict_womens.seed_diff.isna())
                    & (~to_predict_womens.seed_diff.isna())].copy()

preds_sky_mens = make_predictions(to_predict_sky_mens, features_sky_mens, pred_suffix='sky_mens')

preds_sky_womens = make_predictions(to_predict_sky_womens, features_sky_womens, pred_suffix='sky_womens')


In [ ]:
to_predict_sky_mens.Season.unique()

array([2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013,
       2014, 2015, 2016, 2017, 2018, 2019, 2021, 2022, 2023])

In [26]:
final_submission = to_predict[to_predict.type == 'Prediction']

In [27]:
final_submission = final_submission.merge(preds_sky_mens[['ID', 'model_preds_sky_mens']], how='left', on='ID')
final_submission = final_submission.merge(preds_538_mens[['ID', 'model_preds_538_mens']], how='left', on='ID')

final_submission = final_submission.merge(preds_sky_womens[['ID', 'model_preds_sky_womens']], how='left', on='ID')
final_submission = final_submission.merge(preds_538_womens[['ID', 'model_preds_538_womens']], how='left', on='ID')

In [28]:
final_submission['Pred_Blend'] = final_submission.model_preds_sky_mens * .3 + final_submission.model_preds_538_mens * .7


In [29]:
# Submission 1 -- my model for mens, 538 for womens

final_submission['Pred_all_50'] = final_submission['Pred'].copy()

final_submission['Pred'] = np.where(final_submission.model_preds_sky_mens.notnull(), 
                                    final_submission.model_preds_sky_mens, 

                            np.where(final_submission.model_preds_538_womens.notnull(), 
                                    final_submission.model_preds_538_womens,
                                    
                            final_submission['Pred_all_50']))


In [ ]:
# Double checking that we have the right number of predictions to match the number of tourney games
    # The rest are all .5 because they involve non-tourney teams
final_submission[final_submission.Pred != .5].shape[0] / 2 == 68*67/2

True

In [547]:
final_submission[['ID', 'Pred']].to_csv('../submissions/sub1_3_13.csv', index=False)

In [30]:
# Submission 2 -- blend model for mens, 538 for womens

final_submission['Pred'] = np.where(final_submission.Pred_Blend.notnull(), 
                                    final_submission.Pred_Blend, 

                            np.where(final_submission.model_preds_538_womens.notnull(), 
                                    final_submission.model_preds_538_womens,
                                    
                            final_submission['Pred_all_50']))


In [549]:
# Double checking that we have the right number of predictions to match the number of tourney games
    # The rest are all .5 because they involve non-tourney teams
final_submission[final_submission.Pred != .5].shape[0] / 2 == 68*67/2

True

In [ ]:
final_submission[['ID', 'Pred']].to_csv('../submissions/sub2_3_13.csv', index=False)

### Exploring Predictions

In [31]:
sample = final_submission[ (final_submission.Pred_Blend.notnull()) & (final_submission.Gender == 'M')]
sample = add_team_names(sample, mteams)

In [595]:
mteams[mteams.TeamName == 'Duke']

,TeamID,TeamName,FirstD1Season,LastD1Season
80,1181,Duke,1985,2023


In [70]:
sample[
    (sample.Team1_Name.str.contains('Michigan S'))
    & 
    (sample.Team2_Name.str.contains('Tenn'))
    ]

,type,ID,Pred,Season,Team1,Team2,Outcome,Gender,model_preds_sky_mens,model_preds_538_mens,model_preds_sky_womens,model_preds_538_womens,Pred_Blend,Pred_all_50,Team1_Name,Team2_Name
1736,Prediction,2023_1277_1397,0.361215,2023,1277,1397,None,M,0.280708,0.395718,NaN,NaN,0.361215,0.5,Michigan St,Tennessee


In [71]:
experts[experts.ID == '2023_1277_1397']

,ID,Pred
47097,2023_1277_1397,0.32817


In [60]:
experts = pd.read_csv('../data/ExpertsMedianSubmission.csv')

### Validation

In [443]:
features = [
    't1_team_rating_538', 
       't2_team_rating_538']

to_predict_sky_mens_train = to_predict_sky_mens[(to_predict_sky_mens.Season > 2015) &
                                                    (to_predict_sky_mens.Season < 2023)].copy()

mv_538 = ModelValidation(to_predict_sky_mens_train, features, min_training_size=1, select_features=False)
mv_538.validate()

2017
0.17647127894974035
2018
0.19813837766315945
2019
0.15997926893713357
2021
0.2136862356419708
2022
0.20974222489300143


In [444]:
mv.avg_validation_score

0.18443315216610515

In [445]:
features_sky_mens = [
    'seed_diff', 
            't1_adj_margin', 't2_adj_margin',
                't1_final_rank', 't2_final_rank',
                't1_OrdinalRank', 't2_OrdinalRank']

to_predict_sky_mens_train = to_predict_sky_mens[(to_predict_sky_mens.Season < 2023)].copy()

mv_sky = ModelValidation(to_predict_sky_mens_train, features_sky_mens, min_training_size=1, select_features=False)
mv_sky.validate()

2004
0.17788694309542014
2005
0.1716332335959565
2006
0.18972411059924557
2007
0.1437433861771533
2008
0.1599671800701024
2009
0.16679005369571526
2010
0.18762306690446984
2011
0.22194528877063274
2012
0.18767584330594075
2013
0.20350572487780136
2014
0.1987324513380793
2015
0.16563020304894044
2016
0.1822819478938221
2017
0.16656921132099728
2018
0.20557820247556305
2019
0.16761500210361321
2021
0.21730090419547327
2022
0.20559398552096586


In [441]:
mv.avg_validation_score

0.18443315216610515

In [667]:
# Analyzing Ensemble Performance

all_538 = []
all_sky = []
all_combined = []
df_list = []

for preds_538, preds_sky, year in list(zip(mv_538.fold_preds, mv_sky.fold_preds[-5:], [2017, 2018, 2019, 2021, 2022])):
    
    preds_538_pred = preds_538[:,1]
    preds_sky_pred = preds_sky[:,1]
    avg_pred = (preds_538_pred * .7) + (preds_sky_pred * .3)

    tmp_df = to_predict_sky_mens[to_predict_sky_mens.Season == year].copy()
    tmp_df['538_pred'] = preds_538_pred
    tmp_df['sky_pred'] = preds_sky_pred
    df_list.append(tmp_df)

    label = to_predict_sky_mens[to_predict_sky_mens.Season == year].Outcome.astype(int)

    all_538.append(brier_score_loss(label, preds_538_pred))
    all_sky.append(brier_score_loss(label, preds_sky_pred))
    all_combined.append(brier_score_loss(label, avg_pred))


In [671]:
# Analyzing where models to do better / worse
analysis_df = pd.concat(df_list, axis=0)

In [675]:
analysis_df['pred_bucket_sky'] = pd.qcut(analysis_df.sky_pred, q=10)
analysis_df['pred_bucket_538'] = pd.qcut(analysis_df['538_pred'], q=10)

In [696]:
analysis_df



,type,ID,Pred,Season,Team1,Team2,Outcome,Gender,t1_team_rating_538,t1_rd1_win,t1_rd2_win,t1_rd3_win,t1_rd4_win,t1_rd5_win,t1_rd6_win,t1_rd7_win,t2_team_rating_538,t2_rd1_win,t2_rd2_win,t2_rd3_win,t2_rd4_win,t2_rd5_win,t2_rd6_win,t2_rd7_win,t1_OrdinalRank,t2_OrdinalRank,t1_Seed,t2_Seed,seed_diff,t1_adj_oe,t1_adj_de,t1_adj_margin,t2_adj_oe,t2_adj_de,t2_adj_margin,t1_final_rank,t2_final_rank,t1_adjem,t1_adjo,t1_adjo_rank,t1_adjd,t1_adjd_rank,t1_adjt,t1_adjt_rank,t1_luck,t1_luck_rank,t1_sos_adjem,t1_sos_adjem_rank,t1_sos_opp_o,t1_sos_opp_o_rank,t1_sos_opp_d,t1_sos_opp_d_rank,t1_ncsos_adjem_rank,t2_adjem,t2_adjo,t2_adjo_rank,t2_adjd,t2_adjd_rank,t2_adjt,t2_adjt_rank,t2_luck,t2_luck_rank,t2_sos_adjem,t2_sos_adjem_rank,t2_sos_opp_o,t2_sos_opp_o_rank,t2_sos_opp_d,t2_sos_opp_d_rank,t2_ncsos_adjem_rank,538_pred,sky_pred,pred_bucket_sky,pred_bucket_538
1215,Historical,2022_1437_1174,None,2022,1437,1174,1,M,90.22,1.000000,0.948321,0.677927,0.395636,0.194291,1.034951e-01,4.686829e-02,73.57,1.0,0.051679,0.008568,0.001100,0.000119,0.000017,2.138466e-06,5.0,25.0,2.0,15.0,-13.0,122.571306,95.716418,26.854888,108.898266,107.331602,1.566664,90.380569,73.329333,24.2,118.0,8.0,93.8,28.0,62.6,345.0,0.051,71.0,10.8,10.0,108.6,11.0,97.8,15.0,17.0,2.2,106.8,100.0,104.6,212.0,66.6,199.0,0.023,130.0,-2.6,206.0,101.8,205.0,104.4,219.0,270.0,0.915220,0.925951,"(0.856, 0.981]","(0.846, 0.989]"
931,Historical,2017_1452_1137,None,2017,1452,1137,1,M,90.80,1.000000,0.901830,0.645601,0.256006,0.146890,6.777382e-02,3.533303e-02,77.93,1.0,0.098170,0.028442,0.003276,0.000703,0.000115,2.504220e-05,19.0,25.0,4.0,13.0,-9.0,122.356431,93.210870,29.145561,114.661484,103.781375,10.880109,93.380430,77.511765,26.9,116.5,27.0,89.6,5.0,69.6,79.0,-0.043,288.0,9.9,28.0,109.4,38.0,99.5,16.0,341.0,9.0,109.2,85.0,100.2,85.0,69.4,86.0,0.018,144.0,-4.0,257.0,101.1,308.0,105.1,161.0,129.0,0.856810,0.862304,"(0.856, 0.981]","(0.846, 0.989]"
1166,Historical,2021_1116_1331,None,2021,1116,1331,1,M,86.27,1.000000,0.762797,0.403660,0.190939,0.069694,2.747944e-02,1.032973e-02,73.27,1.0,0.056223,0.011256,0.001715,0.000172,0.000021,2.605881e-06,25.0,25.0,3.0,15.0,-12.0,113.632045,93.636639,19.995406,112.213947,109.894043,2.319904,88.227827,72.969157,23.0,112.2,35.0,89.2,14.0,73.1,17.0,0.056,57.0,11.2,60.0,108.2,56.0,97.0,69.0,282.0,2.7,109.4,74.0,106.7,285.0,71.8,47.0,0.013,147.0,0.1,177.0,102.9,173.0,102.8,206.0,4.0,0.886522,0.861609,"(0.856, 0.981]","(0.846, 0.989]"
940,Historical,2017_1257_1240,None,2017,1257,1240,1,M,90.80,1.000000,0.973818,0.668186,0.465179,0.216097,1.108220e-01,4.973675e-02,71.17,1.0,0.026182,0.002149,0.000259,0.000017,0.000002,1.284600e-07,12.0,25.0,2.0,15.0,-13.0,121.750359,94.009978,27.740381,108.306992,106.107608,2.199384,91.364052,70.621020,26.6,117.1,23.0,90.5,7.0,67.7,182.0,-0.011,215.0,12.1,9.0,111.9,4.0,99.8,24.0,48.0,0.8,105.5,150.0,104.7,177.0,63.3,329.0,0.029,104.0,-1.9,191.0,104.1,176.0,106.0,219.0,133.0,0.938701,0.931109,"(0.856, 0.981]","(0.846, 0.989]"
939,Historical,2017_1246_1297,None,2017,1246,1297,1,M,92.28,1.000000,0.966806,0.681923,0.474829,0.302250,1.682376e-01,8.180735e-02,72.83,1.0,0.033194,0.003763,0.000472,0.000062,0.000007,7.006120e-07,2.0,25.0,2.0,15.0,-13.0,124.801663,96.373392,28.428272,111.541837,109.243505,2.298331,92.605689,72.897644,26.9,119.0,15.0,92.1,11.0,72.5,16.0,0.008,169.0,9.5,33.0,109.3,39.0,99.8,25.0,61.0,1.4,106.7,131.0,105.3,187.0,67.5,193.0,0.082,13.0,-3.8,248.0,102.6,257.0,106.4,238.0,228.0,0.937246,0.940892,"(0.856, 0.981]","(0.846, 0.989]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2346,Historical,2019_1328_1438,None,2019,1328,1438,0,M,83.01,1.000000,0.529088,0.062474,0.028344,0.008492,2.822743e-03,7.833322e-04,95.22,1.0,0.981434,0.882199,0.726784,0.493113,0.315200,1.722

In [707]:
scores = []

for index, row in analysis_df[['t1_Seed', 't2_Seed']].drop_duplicates().iterrows():
    
    t1seed = row['t1_Seed']
    t2seed = row['t2_Seed']

    tmp = analysis_df[ (analysis_df.t1_Seed == t1seed)
                      & (analysis_df.t2_Seed == t2seed)]

    scores.append((t1seed, t2seed, brier_score_loss(tmp['Outcome'].astype(int), tmp['sky_pred']), tmp.shape[0]))



In [710]:
# Feels like not enough data to evaluate -- prob driven by 1 or 2 "bad" games
seed_analysis = pd.DataFrame(scores,columns=['seed1', 'seed2', 'score', 'n_records'])
seed_analysis[seed_analysis.n_records > 8].sort_values(by='score', ascending=False)

,seed1,seed2,score,n_records
19,2.0,7.0,0.322052,10
62,7.0,2.0,0.322052,10
76,11.0,3.0,0.298655,11
22,3.0,11.0,0.298655,11
44,11.0,6.0,0.270628,20
18,6.0,11.0,0.270628,20
37,7.0,10.0,0.248830,19
33,10.0,7.0,0.248830,19
43,11.0,11.0,0.243104,18
26,8.0,9.0,0.236987,20


In [ ]:
8

In [ ]:
scores = []

analysis_df.sort_values(by='pred_bucket_538', ascending=False, inplace=True)

for i in list(analysis_df.pred_bucket_538.unique()):
    
    tmp = analysis_df[analysis_df.pred_bucket_538 == i]

    y_true = tmp['Outcome']
    scores.append((i, brier_score_loss(tmp['Outcome'].astype(int), tmp['538_pred'])))

#analysis_df.groupby('pred_bucket_sky').apply()

In [694]:
scores = []

analysis_df.sort_values(by='pred_bucket_538', ascending=False, inplace=True)

for i in list(analysis_df.pred_bucket_538.unique()):
    
    tmp = analysis_df[analysis_df.pred_bucket_538 == i]

    y_true = tmp['Outcome']
    scores.append((i, brier_score_loss(tmp['Outcome'].astype(int), tmp['538_pred'])))

#analysis_df.groupby('pred_bucket_sky').apply()

In [695]:
# 538 scores
scores

[(Interval(0.846, 0.989, closed='right'), 0.07064103326479314),
 (Interval(0.747, 0.846, closed='right'), 0.18184607045330634),
 (Interval(0.663, 0.747, closed='right'), 0.20699599118292322),
 (Interval(0.576, 0.663, closed='right'), 0.2560349501360788),
 (Interval(0.5, 0.576, closed='right'), 0.24228432735985644),
 (Interval(0.424, 0.5, closed='right'), 0.2425112589080959),
 (Interval(0.337, 0.424, closed='right'), 0.2532466317486658),
 (Interval(0.253, 0.337, closed='right'), 0.21047461763848163),
 (Interval(0.154, 0.253, closed='right'), 0.18184607045330634),
 (Interval(0.0104, 0.154, closed='right'), 0.07064103326479315)]

In [690]:
# my scores 
scores

[(Interval(0.856, 0.981, closed='right'), 0.07917498351676285),
 (Interval(0.761, 0.856, closed='right'), 0.14197614302190797),
 (Interval(0.673, 0.761, closed='right'), 0.26080743188302513),
 (Interval(0.585, 0.673, closed='right'), 0.2447312424724296),
 (Interval(0.5, 0.585, closed='right'), 0.23637691274208544),
 (Interval(0.415, 0.5, closed='right'), 0.23637691275264736),
 (Interval(0.327, 0.415, closed='right'), 0.24473124250121037),
 (Interval(0.239, 0.327, closed='right'), 0.260807431873457),
 (Interval(0.144, 0.239, closed='right'), 0.14197614302250933),
 (Interval(0.0183, 0.144, closed='right'), 0.0791749835155887)]

In [680]:
tmp.columns

Index(['type', 'ID', 'Pred', 'Season', 'Team1', 'Team2', 'Outcome', 'Gender',
       't1_team_rating_538', 't1_rd1_win', 't1_rd2_win', 't1_rd3_win',
       't1_rd4_win', 't1_rd5_win', 't1_rd6_win', 't1_rd7_win',
       't2_team_rating_538', 't2_rd1_win', 't2_rd2_win', 't2_rd3_win',
       't2_rd4_win', 't2_rd5_win', 't2_rd6_win', 't2_rd7_win',
       't1_OrdinalRank', 't2_OrdinalRank', 't1_Seed', 't2_Seed', 'seed_diff',
       't1_adj_oe', 't1_adj_de', 't1_adj_margin', 't2_adj_oe', 't2_adj_de',
       't2_adj_margin', 't1_final_rank', 't2_final_rank', 't1_adjem',
       't1_adjo', 't1_adjo_rank', 't1_adjd', 't1_adjd_rank', 't1_adjt',
       't1_adjt_rank', 't1_luck', 't1_luck_rank', 't1_sos_adjem',
       't1_sos_adjem_rank', 't1_sos_opp_o', 't1_sos_opp_o_rank',
       't1_sos_opp_d', 't1_sos_opp_d_rank', 't1_ncsos_adjem_rank', 't2_adjem',
       't2_adjo', 't2_adjo_rank', 't2_adjd', 't2_adjd_rank', 't2_adjt',
       't2_adjt_rank', 't2_luck', 't2_luck_rank', 't2_sos_adjem',
       't2

In [504]:
all_538, all_sky, all_combined

([0.17647127894974035,
  0.19813837766315945,
  0.15997926893713357,
  0.2136862356419708,
  0.20974222489300143],
 [0.16656921132099728,
  0.20557820247556305,
  0.16761500210361321,
  0.21730090419547327,
  0.20559398552096586],
 [0.17290925303179203,
  0.2000121699907515,
  0.16199183925392588,
  0.21393552256484452,
  0.20816963438791772])

In [505]:
sum(all_538) / len(all_538), sum(all_sky) / len(all_sky), sum(all_combined) / len(all_combined)

(0.19160347721700113, 0.19253146112332256, 0.19140368384584633)

In [465]:
preds_sky_pred

array([0.6241359 , 0.58748094, 0.49926096, 0.5346188 , 0.69616435,
       0.94520212, 0.42116916, 0.97040594, 0.9563346 , 0.62537702,
       0.59420173, 0.46529238, 0.28911226, 0.58897124, 0.66571159,
       0.27248762, 0.58176777, 0.10480513, 0.89607676, 0.89759119,
       0.93370153, 0.88899897, 0.90547192, 0.8285099 , 0.81452986,
       0.3382561 , 0.40255378, 0.57531155, 0.30154759, 0.54989222,
       0.90710551, 0.45780008, 0.65249628, 0.85219853, 0.92595115,
       0.76045716, 0.75164891, 0.8404174 , 0.85233204, 0.33599977,
       0.26197595, 0.65061702, 0.28391987, 0.68207151, 0.81237715,
       0.72011894, 0.63040084, 0.33941094, 0.2314216 , 0.59592771,
       0.74070367, 0.71205712, 0.16838667, 0.59988496, 0.51748962,
       0.70675875, 0.78645038, 0.4892466 , 0.29931502, 0.10602863,
       0.62993626, 0.48177623, 0.86491767, 0.78104181, 0.54932815,
       0.30717491, 0.77005183, 0.3758641 , 0.41251906, 0.50073904,
       0.4653812 , 0.30383565, 0.05479788, 0.57883084, 0.02959

In [432]:
features = [
    't1_team_rating_538', 
       't2_team_rating_538']

to_predict_sky_womens_train = to_predict_sky_womens[(to_predict_sky_womens.Season > 2015) &
                                                    (to_predict_sky_womens.Season < 2023)].copy()

mv = ModelValidation(to_predict_sky_womens_train, features, min_training_size=1, select_features=False)
mv.validate()

2017
0.13224826145210217
2018
0.15315700440441862
2019
0.12005883793499716
2021
0.13455375769817304
2022
0.1468279380243678


In [433]:
mv.avg_validation_score

0.13736915990281176

In [434]:
features = ['seed_diff', 
't1_adj_margin', 't2_adj_margin']

to_predict_sky_womens_train = to_predict_sky_womens[
                                            (to_predict_sky_womens.Season > 2015) &
                                            (to_predict_sky_womens.Season < 2023)].copy()

mv = ModelValidation(to_predict_sky_womens_train, features, min_training_size=1, select_features=False)
mv.validate()

2017
0.141771056588384
2018
0.15315191498808117
2019
0.12795348872932316
2021
0.14586764572819175
2022
0.14932257189311743


In [435]:
mv.avg_validation_score

0.1436133355854195

,type,ID,Pred,Season,Team1,Team2,Outcome,Gender,t1_Seed,t2_Seed,seed_diff,t1_adj_oe,t1_adj_de,t1_adj_margin,t2_adj_oe,t2_adj_de,t2_adj_margin,t1_team_rating_538,t1_rd1_win,t1_rd2_win,t1_rd3_win,t1_rd4_win,t1_rd5_win,t1_rd6_win,t1_rd7_win,t2_team_rating_538,t2_rd1_win,t2_rd2_win,t2_rd3_win,t2_rd4_win,t2_rd5_win,t2_rd6_win,t2_rd7_win
0,Historical,2010_3124_3201,None,2010,3124,3201,1,W,4.0,13.0,-9.0,107.882842,74.710405,33.172437,106.154047,84.227119,21.926928,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Historical,2010_3173_3395,None,2010,3173,3395,1,W,8.0,9.0,-1.0,104.868875,84.221074,20.647801,102.122125,82.146304,19.975820,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Historical,2010_3181_3214,None,2010,3181,3214,1,W,2.0,15.0,-13.0,107.154807,72.451354,34.703453,96.652395,86.012954,10.639441,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Historical,2010_3199_3256,None,2010,3199,3256,1,W,3.0,14.0,-11.0,107.639911,79.331522,28.308388,103.644672,89.383894,14.260778,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Historical,2010_3207_3265,None,2010,3207,3265,1,W,5.0,12.0,-7.0,106.185074,85.474293,20.710780,106.940600,89.434223,17.506377,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65806,Prediction,2023_3437_3450,0.5,2023,3437,3450,None,W,4.0,5.0,-1.0,113.722124,84.699900,29.022223,101.331588,85.071535,16.260054,88.55,1.0,0.898477,0.701030,0.141720,0.050785,0.014056,0.001315,86.55,1.0,0.585314,0.167877,0.023924,0.006512,0.001684,0.000147
65808,Prediction,2023_3437_3452,0.5,2023,3437,3452,None,W,4.0,10.0,-6.0,113.722124,84.699900,29.022223,95.601702,82.052837,13.548865,88.55,1.0,0.898477,0.701030,0.141720,0.050785,0.014056,0.001315,81.82,1.0,0.427980,0.053331,0.011778,0.000321,0.000045,0.000008
65877,Prediction,2023_3439_3450,0.5,2023,3439,3450,None,W,1.0,5.0,-4.0,114.751123,82.050175,32.700948,101.331588,85.071535,16.260054,95.72,1.0,0.996053,0.902687,0.525505,0.254243,0.116930,0.023108,86.55,1.0,0.585314,0.167877,0.023924,0.006512,0.001684,0.000147
65879,Prediction,2023_3439_3452,0.5,2023,3439,3452,None,W,1.0,10.0,-9.0,114.751123,82.050175,32.700948,95.601702,82.052837,13.548865,95.72,1.0,0.996053,0.902687,0.525505,0.254243,0.116930,0.023108,81.82,1.0,0.427980,0.053331,0.011778,0.000321,0.000045,0.000008


In [265]:
features = [
    't1_team_rating_538', 
       't2_team_rating_538']


mv = ModelValidation(to_predict_538_train, features, min_training_size=1, select_features=False)
mv.validate()



2017
0.17647127894974035
2018
0.19813837766315945
2019
0.15997926893713357
2021
0.2136862356419708
2022
0.20974222489300143


In [266]:
mv.avg_validation_score

0.19160347721700113

In [ ]:
to_predict_womens = feature_engineering.TournamentSeed(tourney_seeds=seeds_w).add(to_predict_womens)
to_predict_womens = feature_engineering.Efficiency(games=regular_season_games_w, away_bonus=0).add(to_predict_womens)


In [ ]:
sys_counts = []

for s in rankings.SystemName.unique():
    season_count = len(list(rankings[rankings.SystemName == s].Season.unique()))
    sys_counts.append((s, season_count))
    


In [ ]:
df = pd.DataFrame(sys_counts, columns=['system', 'count'])
df[df['count'] == 21]

# could build model using just these rankings to produce ranking prediction

### Building and Testing

In [10]:
mens = to_predict[to_predict.Gender == 'M']

In [11]:
features = [
    'seed_diff', 
            't1_adj_margin', 't2_adj_margin',
                't1_final_rank', 't2_final_rank',
                't1_OrdinalRank', 't2_OrdinalRank']



In [ ]:
tmp = tourney_games
X = tmp[features]
y = tmp['Outcome']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)


In [ ]:
find_best_params(X_train, y_train)

In [11]:
def find_best_params(X_train, y_train):
    clf = LogisticRegression(random_state = 0)
    params = {'C': np.logspace(start=-5, stop=3, num=50), 'penalty': ['l2']}
    clf = GridSearchCV(clf, params, scoring='neg_brier_score', refit=True)
    X_train_scaled = StandardScaler().fit_transform(X_train)
    clf.fit(X_train_scaled, y_train)
    best_params = clf.best_params_
    return best_params


In [28]:
kp_features = ['t1_adjem', 't1_adjo', 't1_adjo_rank', 't1_adjd', 't1_adjd_rank',
       't1_adjt', 't1_adjt_rank', 't1_luck', 't1_luck_rank', 't1_sos_adjem',
       't1_sos_adjem_rank', 't1_sos_opp_o', 't1_sos_opp_o_rank',
       't1_sos_opp_d', 't1_sos_opp_d_rank', 
       't1_ncsos_adjem_rank', 't2_adjem', 't2_adjo', 't2_adjo_rank', 't2_adjd',
       't2_adjd_rank', 't2_adjt', 't2_adjt_rank', 't2_luck', 't2_luck_rank',
       't2_sos_adjem', 't2_sos_adjem_rank', 't2_sos_opp_o',
       't2_sos_opp_o_rank', 't2_sos_opp_d', 't2_sos_opp_d_rank',
      't2_ncsos_adjem_rank']


my_features = [
    'seed_diff', 
            't1_adj_margin', 't2_adj_margin',
              't1_OrdinalRank', 't2_OrdinalRank',
                't1_final_rank', 't2_final_rank']


all_features = kp_features + my_features




In [ ]:
class ParameterFinder():

    def parameter_grid_search(self, X_train, y_train):
            '''gets best parameters using cross validation'''

            clf = LogisticRegression(random_state = 0)
            params = {'C': np.logspace(start=-5, stop=3, num=50), 'penalty': ['l2']}
            clf = GridSearchCV(clf, params, scoring='neg_brier_score', refit=True)
            X_train_scaled = StandardScaler().fit_transform(X_train)
            clf.fit(X_train_scaled, y_train)
            best_params = clf.best_params_
            return best_params


In [23]:
class ModelPrediction:

    def __init__(self):
        self.trained_model=None

    def grid_search_get_best_params(self, X_train, y_train):

        clf = LogisticRegression(random_state = 0)
        params = {'C': np.logspace(start=-5, stop=3, num=50), 'penalty': ['l2']}
        clf = GridSearchCV(clf, params, scoring='neg_brier_score', refit=True)
        X_train_scaled = StandardScaler().fit_transform(X_train)
        clf.fit(X_train_scaled, y_train)
        best_params = clf.best_params_
        return best_params 

    def fit(self, X_train, y_train, params=None):
        ''' train model on unseen data '''

        parameters_to_use = None

        if params is not None: 
            self.params_to_use = params
        
        else:
            self.params_to_use = self.grid_search_get_best_params(X_train, y_train)
        
        lr = Pipeline([('scale', StandardScaler()),('logreg', LogisticRegression(**self.params_to_use))])
        lr.fit(X_train, y_train)
        
        self.trained_model = lr

      
    def predict_proba(self, X_test):
        ''' predict dataset using trained model'''
        preds = self.trained_model.predict_proba(X_test)
        return preds


In [442]:
class ModelValidation:

    def __init__(self, tourney_games, available_features, min_training_size=5, select_features=True):
        self.tourney_games=tourney_games
        self.available_features=available_features
        self.select_features=select_features
        self.min_training_size = min_training_size

    def grid_search_get_best_params(self, X_train, y_train):

        clf = LogisticRegression(random_state = 0)
        params = {'C': np.logspace(start=-5, stop=3, num=50), 'penalty': ['l2']}
        clf = GridSearchCV(clf, params, scoring='neg_brier_score', refit=True)
        X_train_scaled = StandardScaler().fit_transform(X_train)
        clf.fit(X_train_scaled, y_train)
        best_params = clf.best_params_
        return best_params 


    def hyperopt_get_best_params(self, X_train, y_train):

        hp_search_space = {'C': hp.uniform('C', .0001, 1000)}
        oms = OptimizedModelSelector(
                              n_rounds=5
                            , X=X_train
                            , y=y_train
                            , starting_model=LogisticRegression
                            , starting_features=self.available_features
                            , hyper_parameter_space=hp_search_space)
        oms.run()

        return oms.features, oms.selected_params
                
    def validate(self):
        ''' returns estimate for model performance using shifted validation'''

        scores = []
        tourney_games = self.tourney_games
        self.fold_preds = []
        
        for n, season in enumerate(tourney_games.Season.unique()):
            
            if n >= self.min_training_size and n < len(tourney_games.Season.unique()):

                print(season)

                train = tourney_games[tourney_games.Season < season]
                test = tourney_games[tourney_games.Season == season]

                X_train = train[self.available_features]
                X_test = test[self.available_features]
                y_train = train['Outcome'].astype(int)
                y_test = test['Outcome'].astype(int)

                if self.select_features:
                    features, params = self.hyperopt_get_best_params(X_train, y_train)
                    print(features)

                else:
                    features=self.available_features
                    params=self.grid_search_get_best_params(X_train, y_train)

                lr = Pipeline([('scale', StandardScaler()),('logreg', LogisticRegression(**params))])
                lr.fit(X_train[features], y_train)

                y_prob = lr.predict_proba(X_test[features])
                self.fold_preds.append(y_prob)
                loss = brier_score_loss(y_test, y_prob[:,1])
                print(loss)
                scores.append((season, loss))

        self.validation_df = pd.DataFrame(scores, columns = ['season', 'score']).sort_values(by = 'score')
        self.avg_validation_score = self.validation_df.score.mean()


In [14]:
from sklearn.model_selection import cross_val_score

class OptimizedModelSelector:

    def __init__(self, n_rounds, X, y, starting_model, starting_features, hyper_parameter_space, cv_folds=5, default_feature='seed_diff'):
        self.starting_model=starting_model
        self.starting_features=starting_features
        self.hyper_parameter_space=hyper_parameter_space
        self.X = X.copy()
        self.y = y.copy()
        self.folds=cv_folds
        self.n_rounds=n_rounds
        self.default_feature='seed_diff' # feature to use in case model selector tries to be stupid and use 0 features
        self.round_metrics = {} # key is the loss, value is a dict of features and hyperparams

    def objective(self, params):

        selected_features = []
        param_dict = {}

        for i, j in params.items():
            if ('_var' in i) and (j == 1):
                selected_features.append(i)
            elif ('_var' not in i):
                param_dict[i] = j + .000000000001 # avoid divide by zero

        if len(selected_features) == 0:
            selected_features = [self.default_feature + '_var']

        model = Pipeline([('scale', StandardScaler()), ('model', self.starting_model(**param_dict, random_state=0,
                                                                                max_iter=1000))])
       
        cv_scores = cross_val_score(model, self.X[selected_features], self.y, scoring='neg_brier_score', cv=self.folds,
                                error_score='raise', verbose=0)

        return -1 * (sum(cv_scores) / len(cv_scores))
    
    def get_features_and_params_from_best(self, best):

        self.features = []
        self.selected_params = {}
        
        for k,v in best.items():
            if v == 1 and k not in self.hyper_parameter_space.keys():
                self.features.append(k)
            if k in self.hyper_parameter_space.keys():
                self.selected_params[k] = v

    def run(self):

        search_space = self.hyper_parameter_space
        self.features = self.starting_features

        for round in range(self.n_rounds):

            search_space = {}
            
            for k,v in self.hyper_parameter_space.items():
                search_space[k] = v
    
            for col in self.features:
                self.X.rename(columns={col:col+'_var'}, inplace=True)
                search_space[col + '_var'] = hp.choice(col, [0,1])
            
            trials = Trials()

            # print(search_space)

            best = fmin(self.objective, search_space, algo=tpe.suggest,
                                max_evals=100, trials=trials)

            best_loss = trials.best_trial['result']['loss']
            self.get_features_and_params_from_best(best)

            self.round_metrics[best_loss] = {'features': self.features,
                                            'params': self.selected_params}

        # find best score across all rounds, set features / parameters associated with round
        best_score = min(self.round_metrics.keys())
        self.features = self.round_metrics[best_score]['features']
        self.selected_params = self.round_metrics[best_score]['params']




In [21]:
t = to_predict_mens[ (to_predict_mens.Season > 2010)
                & (to_predict_mens.Season < 2023)]

hp_search_space = {'C': hp.quniform('C', .0001, 1000, 1)}

all_features = my_features.copy()

oms = OptimizedModelSelector(X=t
                       , n_rounds = 2
                       , y=t['Outcome'].astype(int)
                       , starting_model=LogisticRegression
                       , starting_features=all_features
                       , hyper_parameter_space=hp_search_space)

In [61]:
t = to_predict_mens[ (to_predict_mens.Season > 2010)
                & (to_predict_mens.Season < 2023)]


features = all_features
# selected = ['t1_OrdinalRank', 't1_adj_margin', 't1_ncsos_adjem_rank', 't1_sos_adjem_rank', 't1_sos_opp_o_rank', 't2_OrdinalRank', 't2_adj_margin', 't2_adjt_rank', 't2_sos_opp_d']

selected = ['t1_OrdinalRank',
't2_OrdinalRank',
't1_adj_margin', 
't2_adj_margin',
't1_sos_adjem_rank',
't2_sos_adjem_rank',
't1_ncsos_adjem_rank',
't2_ncsos_adjem_rank',
't2_adjt_rank',
't1_adjt_rank',
't1_final_rank',
't2_final_rank',
'seed_diff'
]


mv = ModelValidation(t, selected, select_features=False)
mv.validate()


2016
0.18268158770868162
2017
0.16923359890100131
2018
0.2040348063590193
2019
0.16566891638337472
2021
0.21447002156405004
2022
0.1996798928855313


In [27]:
mv.validation_df

,season,score
2,2019,0.162732
0,2017,0.175715
4,2022,0.213336
3,2021,0.216897
1,2018,0.220865


In [ ]:
(0.220865 + 0.216897 + 0.213336 + 0.175715 + 0.162732) / 5

In [62]:
mv.avg_validation_score

0.18929480396694306

In [24]:
mv.avg_validation_score

0.19101405031933438

In [ ]:
tmp = t[['Season', 'Team1', 'Team2','t1_adjem', 't1_adj_margin', 't2_adjem', 't2_adj_margin']]

In [ ]:
tmp['t1_diff'] = abs(tmp['t1_adjem'] - tmp['t1_adj_margin'])
tmp['t2_diff'] = abs(tmp['t2_adjem'] - tmp['t2_adj_margin'])

In [ ]:
tmp.sort_values(by = 't1_diff', ascending=False)[['Season', 'Team1', 't1_adj_margin', 
                                                  't1_adjem', 't1_diff']].drop_duplicates()

In [ ]:
mteams[mteams.TeamID == 1276]

In [ ]:
tmp[tmp.Season == 2021].sort_values(by = 't1_adj_margin', ascending = False).drop_duplicates(subset=['Team1'])

In [ ]:
t[t.Outcome == 'unknown']

In [ ]:
features_w = ['seed_diff',
 't1_adj_margin',
 't2_adj_margin']

In [ ]:
mv = ModelValidation(tourney_games_w, features_w)
mv.validate()

In [ ]:
mv.avg_validation_score

In [ ]:
scores_df = pd.DataFrame(loss_score, columns = ['Season', 'Score']).sort_values(by = 'Score')
scores_df

In [ ]:
# just seeds
scores_df.Score.mean()

In [ ]:
# just seeds
scores_df.Score.mean()

In [ ]:
# add efficiency
scores_df.Score.mean()

In [ ]:
# add ordinal rank
scores_df.Score.mean()

In [ ]:
# all
scores_df.Score.mean()

In [ ]:
X = to_pred_games[feature_list].values
y = to_pred_games['Outcome'].values.ravel()
ss = StandardScaler()

# X = ss.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

#Grid search to get best params
clf = LogisticRegression(random_state = 0)
params = {'C': np.logspace(start=-5, stop=3, num=9), 'penalty': ['l2', 'l1']}
clf = GridSearchCV(clf, params, scoring='neg_log_loss', refit=True)
X_train_scaled = StandardScaler().fit_transform(X_train)
clf.fit(X_train_scaled, y_train)
best_param = clf.best_params_


In [ ]:
#Cross validation
seasons = list(to_pred_games.Season.unique())

log_loss_list = []

for test_season in seasons:
    
    train_seasons = seasons.copy()
    train_seasons.remove(test_season)
 
    X_train = to_pred_games[to_pred_games['Season'].isin(train_seasons)][feature_list].values
    X_test = to_pred_games[to_pred_games.Season == test_season][feature_list].values

    y_train = to_pred_games[to_pred_games['Season'].isin(train_seasons)]['Outcome'].values.ravel()
    y_test = to_pred_games[to_pred_games.Season == test_season]['Outcome'].values.ravel()
    
    logreg = Pipeline([('scale', StandardScaler()),('logreg', LogisticRegression(**best_param))])
    logreg.fit(X_train, y_train)

    #Evaluate score on test set 
    y_pred = logreg.predict_proba(X_test)
    
    ll = log_loss(y_test, y_pred)
    log_loss_list.append(ll)
    
    
print('avg. log loss: {}'.format(sum(log_loss_list) / len(log_loss_list)))
print('min log loss: {}'.format(min(log_loss_list)))
print('max log loss: {}'.format(max(log_loss_list)))
print('std dev log loss: {}'.format(statistics.stdev(log_loss_list)))
print(log_loss_list)
    

In [ ]:
to_predict=SubmissionSetup(sub, tourney_games, tourney_games_w, mteams).setup()

In [ ]:
to_predict.groupby(['type', 'Gender']).count()

In [ ]:
to_predict[to_predict.type == ]

In [ ]:
sub.merge(mteams, how='left', )

In [ ]:
tourney_games.head()

### Create data

### Set up

In [ ]:
pd.set_option('max_columns', 1000)
all_games.head()

In [ ]:
all_games = all_games.fillna(0)

#remove tournament games we are predicting on 
# to_pred_games = all_games[(all_games.Season < 2016)] #this is for the first stage
to_pred_games = all_games[(all_games.Season < 2022)] #this is for the second stage




In [ ]:

feature_list = [
    'seed_diff', 
            't1_adj_margin', 't2_adj_margin',
                't1_final_rank', 't2_final_rank',
                't1_OrdinalRank', 't2_OrdinalRank',
     't1_adj_margin_3',
 't2_adj_margin_3',
     't1_adj_margin_2',
 't2_adj_margin_2',
     't1_adj_margin_10',
 't2_adj_margin_10',
      't1_num_wins',
    't2_num_wins',
     't1_adj_margin_7',
 't2_adj_margin_7',
     't1_won_conf_tournament',
    't2_won_conf_tournament',
      't1_num_losses',
    't2_num_losses',
     't1_adj_margin_5',
 't2_adj_margin_5',
    'num_wins_diff',
]

feature_list = [
    'seed_diff', 
            't1_adj_margin', 't2_adj_margin',
                't1_final_rank', 't2_final_rank',
                't1_OrdinalRank', 't2_OrdinalRank',
]


           

#### Cross validation

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import log_loss
from sklearn.feature_selection import chi2
from sklearn.metrics import r2_score
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline


In [ ]:
import warnings
warnings.filterwarnings('ignore') 

In [ ]:
X = to_pred_games[feature_list].values
y = to_pred_games['Outcome'].values.ravel()
ss = StandardScaler()

# X = ss.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

#Grid search to get best params
clf = LogisticRegression(random_state = 0)
params = {'C': np.logspace(start=-5, stop=3, num=9), 'penalty': ['l2', 'l1']}
clf = GridSearchCV(clf, params, scoring='neg_log_loss', refit=True)
X_train_scaled = StandardScaler().fit_transform(X_train)
clf.fit(X_train_scaled, y_train)
best_param = clf.best_params_


#### Logistic Regression

In [ ]:
#Grid search to get best params
clf = LogisticRegression(random_state = 0)
params = {'C': np.logspace(start=-5, stop=3, num=9), 'penalty': ['l2', 'l1']}
clf = GridSearchCV(clf, params, scoring='neg_log_loss', refit=True)
X_train_scaled = StandardScaler().fit_transform(X_train)
clf.fit(X_train_scaled, y_train)
best_param = clf.best_params_

# #Use best params to train final model
# logreg = Pipeline([('scale', StandardScaler()),('logreg', LogisticRegression(**best_param))])
# logreg.fit(X_train, y_train)

#Evaluate score on test set 
#y_pred = logreg.predict_proba(X_test)
#log_loss(y_test, y_pred)



In [ ]:
LogisticRegression(**best_param)

In [ ]:
import statistics
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler


In [ ]:
seasons = list(all_games.Season.unique())
scores = [0.5223775658655162, 0.5035419334769029, 0.4938799971523059, 0.583826101324787, 0.4297945006936753, 0.4833622866501298, 0.5105272087220717, 0.5053560596048409, 0.6138812776789054, 0.5821446004893247, 0.5625780606741233, 0.5374232777449158, 0.5058912971422388]
pd.DataFrame(list(zip(seasons, scores)))


In [ ]:
#Cross validation
seasons = list(to_pred_games.Season.unique())

log_loss_list = []

for test_season in seasons:
    
    train_seasons = seasons.copy()
    train_seasons.remove(test_season)
 
    X_train = to_pred_games[to_pred_games['Season'].isin(train_seasons)][feature_list].values
    X_test = to_pred_games[to_pred_games.Season == test_season][feature_list].values

    y_train = to_pred_games[to_pred_games['Season'].isin(train_seasons)]['Outcome'].values.ravel()
    y_test = to_pred_games[to_pred_games.Season == test_season]['Outcome'].values.ravel()
    
    logreg = Pipeline([('scale', StandardScaler()),('logreg', LogisticRegression(**best_param))])
    logreg.fit(X_train, y_train)

    #Evaluate score on test set 
    y_pred = logreg.predict_proba(X_test)
    
    ll = log_loss(y_test, y_pred)
    log_loss_list.append(ll)
    
    
print('avg. log loss: {}'.format(sum(log_loss_list) / len(log_loss_list)))
print('min log loss: {}'.format(min(log_loss_list)))
print('max log loss: {}'.format(max(log_loss_list)))
print('std dev log loss: {}'.format(statistics.stdev(log_loss_list)))
print(log_loss_list)
    

In [ ]:
#fit on entire dataset
logreg = Pipeline([('scale', StandardScaler()),('logreg', LogisticRegression(**best_param))])
logreg.fit(X, y)

avg. log loss: 0.5377196208726289
min log loss: 0.4527681615412048
max log loss: 0.6301372973190598
std dev log loss: 0.04826156092665253
[0.5229902686047858, 0.5236486146281775, 0.5158111793350595, 0.56923309326896, 0.4527681615412048, 0.483523972814316, 0.5005711494650689, 0.5390678057159176, 0.6301372973190598, 0.562545719196487, 0.5826151696256913, 0.5697230189568193]


In [ ]:
from numpy import loadtxt
from xgboost import XGBClassifier

In [ ]:
#Grid search to get best params
clf = XGBClassifier(random_state = 0)

params = {'min_child_weight': [5],
        'gamma': [5, 10],
        'subsample': [0.8],
        'colsample_bytree': [0.4, 0.6],
        'max_depth': [1, 2, 3, 10, 20, 30, 50]
        }

clf = GridSearchCV(clf, params, scoring='neg_log_loss', refit=True)
clf.fit(X_train, y_train)
best_param = clf.best_params_


In [ ]:
best_param = clf.best_params_


In [ ]:
best_param = {'colsample_bytree': 0.8,                 
              'learning_rate': 0.0003,
              'max_depth': 5,
              'subsample': 1,
              'objective':'binary:logistic',
              'eval_metric':'logloss',
              'min_child_weight':3,
              'gamma':0.25,
              'n_estimators':500,
              'verbosity':2
              }

In [ ]:
#Cross validation
seasons = list(to_pred_games.Season.unique())

log_loss_list = []

for test_season in seasons:
    
    train_seasons = seasons.copy()
    train_seasons.remove(test_season)
 
    X_train = to_pred_games[to_pred_games['Season'].isin(train_seasons)][feature_list].values
    X_test = to_pred_games[to_pred_games.Season == test_season][feature_list].values

    y_train = to_pred_games[to_pred_games['Season'].isin(train_seasons)]['Outcome'].values.ravel()
    y_test = to_pred_games[to_pred_games.Season == test_season]['Outcome'].values.ravel()
    
    clf = XGBClassifier(**best_param)
    clf.fit(X_train, y_train)

    #Evaluate score on test set 
    y_pred = clf.predict_proba(X_test)
    
    ll = log_loss(y_test, y_pred)
    log_loss_list.append(ll)
    
    
print('avg. log loss: {}'.format(sum(log_loss_list) / len(log_loss_list)))
print('min log loss: {}'.format(min(log_loss_list)))
print('max log loss: {}'.format(max(log_loss_list)))
print('std dev log loss: {}'.format(statistics.stdev(log_loss_list)))
print(log_loss_list)
    

#### SVC

In [ ]:
''' 
from sklearn import svm
Cs = [0.001, 0.01, 0.1, 1, 10]
gammas = [0.001, 0.01, 0.1, 1]
param_grid = {'C': Cs, 'gamma' : gammas}
clf = GridSearchCV(svm.SVC(kernel='rbf', probability = True), param_grid, cv=5)
clf.fit(X, y)
clf.fit(X_train, y_train)
best_param = clf.best_params_


svc_ = svm.SVC(**best_param, probability = True)
svc_.fit(X_train, y_train)

y_pred = svc_.predict_proba(X_test)
log_loss(y_test, y_pred)
''' 


#### Random Forest

In [ ]:
best_param = {'max_depth':10, 'min_samples_leaf':5, 'n_estimators':1000}

In [ ]:
#Grid search to get best params
clf = RandomForestClassifier(random_state = 0)

params = {'min_samples_leaf': [1, 5, 15, 25, 50, 80, 100], 
          'bootstrap': [True, False],
          'max_depth': [1, 3, 5, 10, 20, None],
          'max_features': ['auto', 'sqrt'],
          'min_samples_split': [2, 5, 10],
         }

clf = GridSearchCV(clf, params, scoring='neg_log_loss', refit=True, verbose = 100)
clf.fit(X_train, y_train)
best_param = clf.best_params_



In [ ]:
#Cross validation
seasons = list(to_pred_games.Season.unique())

log_loss_list = []

for test_season in seasons:
    
    train_seasons = seasons.copy()
    train_seasons.remove(test_season)
 
    X_train = to_pred_games[to_pred_games['Season'].isin(train_seasons)][feature_list].values
    X_test = to_pred_games[to_pred_games.Season == test_season][feature_list].values

    y_train = to_pred_games[to_pred_games['Season'].isin(train_seasons)]['Outcome'].values.ravel()
    y_test = to_pred_games[to_pred_games.Season == test_season]['Outcome'].values.ravel()
    
    rf = Pipeline([('scale', StandardScaler()),('logreg', RandomForestClassifier(**best_param))])
    rf.fit(X_train, y_train)

    #Evaluate score on test set 
    y_pred = rf.predict_proba(X_test)
    
    ll = log_loss(y_test, y_pred)
    print(ll)
    log_loss_list.append(ll)
    
    
print('avg. log loss: {}'.format(sum(log_loss_list) / len(log_loss_list)))
print('min log loss: {}'.format(min(log_loss_list)))
print('max log loss: {}'.format(max(log_loss_list)))
print('std dev log loss: {}'.format(statistics.stdev(log_loss_list)))
print(log_loss_list)
    

In [ ]:
#0.516114759994771


# feature_list = ['seed_diff',  
#                 't1_adj_margin','t2_adj_margin',
#                 't1_final_rank', 't2_final_rank',
#                 't1_OrdinalRank', 't2_OrdinalRank',
#                 't1_adj_margin_1',
#        't1_adj_margin_3', 't1_adj_margin_4','t1_adj_margin_10',
#        't2_adj_margin_1', 't2_adj_margin_3',
#        't2_adj_margin_4', 
#        't2_adj_margin_10']

In [ ]:
best_param['n_estimators'] = 1000

In [ ]:
#fit on entire dataset
rf = RandomForestClassifier(**best_param)
rf.fit(X, y)

In [ ]:
best_param

### Output model and data

In [ ]:
#all_games.to_csv('output/all_games.csv')

In [ ]:
import pickle
    
with open("model/logreg.pkl", "wb") as file: 
    pickle.dump(logreg, file)

# with open("model/rf.pkl", "wb") as file: 
#     pickle.dump(rf, file)  

    

In [ ]:
logreg